# Library


In [1]:
import pandas as pd
import os
import pyexasol

# folder

In [2]:
raw1_fold = 'D:/data/big2/BD1/raw'
raw3_fold = 'D:/data/big2/BD3/raw'

# server

In [3]:
# # insider db
# wd = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='VSYSD')
# we = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='vsyse')

In [4]:
# exasol db
wd = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSD')
we = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='vsyse')
wbt = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='wbt')

# Load

## RH에서 제공한 법정동코드

In [5]:
name = 'tb_mapdata_gu(2023.04.20)'
file_name = f'{name}.csv'
df = pd.read_csv(os.path.join(raw1_fold, file_name), low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   dong_code    278 non-null    int64  
 1   ctprvn_code  278 non-null    int64  
 2   signgu_code  278 non-null    int64  
 3   sdstrd_code  278 non-null    int64  
 4   ctprvn_nm    278 non-null    object 
 5   signgu_nm    260 non-null    object 
 6   sdstrd_nm    0 non-null      float64
dtypes: float64(1), int64(4), object(2)
memory usage: 15.3+ KB


In [6]:
df.head()

,dong_code,ctprvn_code,signgu_code,sdstrd_code,ctprvn_nm,signgu_nm,sdstrd_nm
0,0,0,0,0,전국,NaN,NaN
1,1100000000,11,0,0,서울특별시,NaN,NaN
2,1111000000,11,110,0,서울특별시,종로구,NaN
3,1114000000,11,140,0,서울특별시,중구,NaN
4,1117000000,11,170,0,서울특별시,용산구,NaN


In [8]:
TB_MAPDATA = df.copy()

### [upload] TB_MAPDATA

In [9]:
expdf = TB_MAPDATA
table_nm = 'TB_MAPDATA'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 1s
we.import_from_pandas(expdf, table_nm)

## 4등급 result(GRD4_RESULT)

In [10]:
# 9.9s
name = '4등급_차량_리스트_양식_result_최종(전체)(2023.07.03)'
file_name = f'{name}.csv'
df = pd.read_csv(os.path.join(raw1_fold, file_name), low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1502235 entries, 0 to 1502234
Data columns (total 28 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   차대번호      1502235 non-null  object 
 1   차량번호      1502235 non-null  object 
 2   제원관리번호    1502235 non-null  object 
 3   차종        1502235 non-null  object 
 4   용도        1502235 non-null  object 
 5   최초등록일     1502235 non-null  int64  
 6   연식        1502235 non-null  int64  
 7   제작일자      1335178 non-null  float64
 8   검사유효일     1502223 non-null  float64
 9   차명        1502235 non-null  object 
 10  차종분류      1502235 non-null  object 
 11  차종유형      1502235 non-null  object 
 12  자동차형식     1501958 non-null  object 
 13  제작사명      1492590 non-null  object 
 14  연료        1502235 non-null  object 
 15  엔진형식      1500400 non-null  object 
 16  총중량       1502234 non-null  float64
 17  적재중량      1501728 non-null  float64
 18  엔진출력      1501882 non-null  object 
 19  배기량       1502231 non

In [11]:
df.head()

,차대번호,차량번호,제원관리번호,차종,용도,최초등록일,연식,제작일자,검사유효일,차명,...,엔진출력,배기량,법정동코드,시도명,시군구명,소유자구분,말소여부,배인번호_수정,등급_수정,DPF유무_수정
0,WAUZZZ4F16N115327,21로7811,02420000100031305,승용,자가용,20060208,2006,20060119.0,20200207.0,아우디 A6 3.0 TDI Quattro,...,225.0,2967.0,3017011600,대전광역시,서구,주민,N,4MY-AD-20,4.0,무
1,WAUZZZ4F16N113691,23거2347,02420000100031305,승용,자가용,20060228,2006,20060116.0,20220227.0,아우디 A6 3.0 TDI Quattro,...,225.0,2967.0,2635010700,부산광역시,해운대구,외국인,N,4MY-AD-20,4.0,무
2,WAUZZZ4F86N088157,04어9316,02420000100031305,승용,자가용,20060105,2006,20051212.0,20200104.0,아우디 A6 3.0 TDI Quattro,...,225.0,2967.0,4148012200,경기도,파주시,주민,N,4MY-AD-20,4.0,무
3,WAUZZZ4F36N090463,14우2622,02420000100031305,승용,자가용,20060105,2006,20051206.0,20100104.0,아우디 A6 3.0 TDI Quattro,...,225.0,2967.0,2623010900,부산광역시,부산진구,주민,N,4MY-AD-20,4.0,무
4,WAUZZZ4F26N152385,07머3269,02420000100031305,승용,자가용,20060630,2006,20060612.0,20220629.0,아우디 A6 3.0 TDI Quattro,...,225.0,2967.0,5013010300,제주특별자치도,서귀포시,주민,N,4MY-AD-20,4.0,무


In [12]:
GRD4_RESULT = df.copy()

### [upload] GRD4_RESULT

In [13]:
expdf = GRD4_RESULT
table_nm = 'GRD4_RESULT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 1s
we.import_from_pandas(expdf, table_nm)

## 5등급 조기폐차(GRD5_ELPDSRC)

In [15]:
name = '5등급_조기폐차'
file_name = f'{name}.csv'
df = pd.read_csv(os.path.join(raw3_fold, file_name), low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   RGN         68 non-null     object 
 1   YR          68 non-null     int64  
 2   VHCL_MKCNT  68 non-null     int64  
 3   VHCL_REDE   68 non-null     float64
 4   ELPDSRC     68 non-null     float64
 5   NTRL_DCLN   68 non-null     float64
dtypes: float64(3), int64(2), object(1)
memory usage: 3.3+ KB


In [16]:
df.head()

,RGN,YR,VHCL_MKCNT,VHCL_REDE,ELPDSRC,NTRL_DCLN
0,서울,2019,199898,46949.666667,31792.000000,15157.666667
1,서울,2020,161864,38034.000000,23045.000000,14989.000000
2,서울,2021,138381,23483.000000,8148.000000,15335.000000
3,서울,2022,120996,17385.000000,2476.000000,14909.000000
4,인천,2019,90619,20596.666667,16421.666667,4175.000000


In [17]:
GRD5_ELPDSRC = df.copy()

### [upload] GRD5_ELPDSRC

In [18]:
expdf = GRD5_ELPDSRC
table_nm = 'GRD5_ELPDSRC'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 1s
we.import_from_pandas(expdf, table_nm)

## 5등급 저공해 미조치(GRD5_LEM_N_MOD)

In [19]:
name = '5등급_저공해미조치'
file_name = f'{name}.csv'
df = pd.read_csv(os.path.join(raw3_fold, file_name), low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   지역      12 non-null     object
 1   구분      12 non-null     object
 2   차량대수    12 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 416.0+ bytes


In [20]:
df.head()

,지역,구분,차량대수
0,수도권,2차계절제_시작,539748
1,수도권,2차계절제_종료,515615
2,수도권,3차계절제_시작,459110
3,수도권,3차계절제_종료,436921
4,수도권,4차계절제_시작,394533


In [21]:
GRD5_LEM_N_MOD = df.copy()

### [upload] GRD5_LEM_N_MOD

In [22]:
expdf = GRD5_LEM_N_MOD
table_nm = 'GRD5_LEM_N_MOD'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 1s
we.import_from_pandas(expdf, table_nm)

## 5등급 운행제한 건수(GRD5_LMT_NOCS)

In [23]:
name = '5등급_운행제한건수(20230706)'
file_name = f'{name}.csv'
df = pd.read_csv(os.path.join(raw3_fold, file_name), low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 408196 entries, 0 to 408195
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   차대번호    408196 non-null  object
 1   계절제_1차  408196 non-null  int64 
 2   계절제_2차  408196 non-null  int64 
 3   계절제_3차  408196 non-null  int64 
 4   계절제_4차  408196 non-null  int64 
 5   비상시     408196 non-null  int64 
 6   상시      408196 non-null  int64 
dtypes: int64(6), object(1)
memory usage: 21.8+ MB


In [24]:
df.head()

,차대번호,계절제_1차,계절제_2차,계절제_3차,계절제_4차,비상시,상시
0,000000JN44C000673,0,0,9,0,1,0
1,01451201380,0,0,0,0,0,0
2,01751300580,0,1,0,0,0,0
3,01761300921,0,0,0,0,0,0
4,01761301128,0,0,1,0,0,0


In [25]:
GRD5_LMT_NOCS = df.copy()

### [upload] GRD5_LMT_NOCS

In [26]:
expdf = GRD5_LMT_NOCS
table_nm = 'GRD5_LMT_NOCS'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 1s
we.import_from_pandas(expdf, table_nm)

# code end